# NumPy (2)

(Thư viện NumPy - Phần 2)

In [153]:
import numpy as np
np.__version__

'1.25.2'

## Dữ liệu `Iris Data Set`

Dữ liệu `Iris Data Set`: https://archive.ics.uci.edu/ml/datasets/Iris.

Tập tin dữ liệu: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data.

Tham khảo thêm: https://en.wikipedia.org/wiki/Iris_flower_data_set.

In [154]:
iris_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
iris_dtypes = ["f4", "f4", "f4", "f4", "U20"]
iris_table = np.loadtxt("iris.data", delimiter = ",", skiprows = 0, 
                        dtype={"names": iris_names, "formats": iris_dtypes}) 

In [78]:
iris_table[:5]

array([(5.1, 3.5, 1.4, 0.2, 'Iris-setosa'),
       (4.9, 3. , 1.4, 0.2, 'Iris-setosa'),
       (4.7, 3.2, 1.3, 0.2, 'Iris-setosa'),
       (4.6, 3.1, 1.5, 0.2, 'Iris-setosa'),
       (5. , 3.6, 1.4, 0.2, 'Iris-setosa')],
      dtype=[('sepal_length', '<f4'), ('sepal_width', '<f4'), ('petal_length', '<f4'), ('petal_width', '<f4'), ('class', '<U20')])

Trích riêng các features (các cột số đo) và nhãn lớp (class)

In [79]:
iris_classes = iris_table["class"]
iris_features = np.hstack([iris_table[n].reshape(-1, 1) for n in iris_names if n != "class"])

In [80]:
print(iris_classes[:5])
print(iris_features[:5])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


## Broadcasting

**Broadcasting** là kỹ thuật của NumPy giúp thao tác với các mảng (`ndarray`) có số chiều (`ndim`) và/hoặc kích thước (`shape`) khác nhau.

Cùng với vector hóa, broadcasting mang lại sự hiệu quả cho NumPy.

In [81]:
# hàm tự cộng số (scalar) c vào các phần tử của vector vec
def myadd_scalar(vec, c):
    vec_copy = vec.copy()
    for i in range(vec.size):
        vec_copy[i] += c
    return vec_copy

vec = np.arange(10)
print(vec)
print(myadd_scalar(vec, 100))       # tự cộng
print(vec + np.full_like(vec, 100)) # cộng bằng cách tạo mảng tạm và vector hóa
print(vec + 100)                    # cộng bằng broadcasting của NumPy

[0 1 2 3 4 5 6 7 8 9]
[100 101 102 103 104 105 106 107 108 109]
[100 101 102 103 104 105 106 107 108 109]
[100 101 102 103 104 105 106 107 108 109]


In [82]:
vec = np.arange(100_000)
%timeit myadd_scalar(vec, 100)
%timeit (vec + np.full_like(vec, 100))
%timeit (vec + 100)

10.3 ms ± 70.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
48.5 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
15.9 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Các **qui tắc broadcasting**

1. Nếu 2 mảng khác số chiều (`ndim`) thì mảng ít chiều hơn sẽ được thêm chiều với kích thước 1.
2. Nếu 2 mảng cùng số chiều nhưng khác kích thước thì mảng có kích thước 1 sẽ được "độn thêm" để cùng kích thước ở tất cả các chiều.
3. Nếu sau khi thực hiện các qui tắc 1, 2 mà 2 mảng vẫn không cùng `shape` thì không broadcasting được.

Xem thêm https://numpy.org/doc/stable/user/basics.broadcasting.html.

In [83]:
# Ví dụ 1
a = np.array([1, 2, 3])
b = 2
b_like_a = np.full_like(a, b)

print(a)
print(b)
print(b_like_a)
print(a + b_like_a)
print(a + b) # broadcasting

[1 2 3]
2
[2 2 2]
[3 4 5]
[3 4 5]


In [84]:
# Ví dụ 2
a = np.array([[10*i for _ in range(3)] for i in range(4)])
b = np.array([1, 2, 3])
b_like_a = np.full_like(a, b)

print(a)
print(b)
print(b_like_a)
print(a + b_like_a)
print(a + b) # broadcasting

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]]
[1 2 3]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]


In [85]:
# Ví dụ 3
a = np.array([[10*i for _ in range(3)] for i in range(4)])
b = np.array([1, 2, 3, 4])

print(a, a.shape)
print(b, b.shape)
    
try:
    print(a + b) # không broadcasting được
except Exception as e:
    print(e)

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]] (4, 3)
[1 2 3 4] (4,)
operands could not be broadcast together with shapes (4,3) (4,) 


In [86]:
# Ví dụ 4
a = np.array([0, 10, 20, 30]).reshape(-1, 1)
b = np.array([1, 2, 3])

a_like_b = np.full((a.shape[0], b.shape[0]), a)
b_like_a = np.full((a.shape[0], b.shape[0]), b)

print(a)
print(b)
print(a_like_b)
print(b_like_a)
print(a_like_b + b_like_a)
print(a + b) # broadcasting

[[ 0]
 [10]
 [20]
 [30]]
[1 2 3]
[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]
[[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]


**Bài tập**

Từ ma trận `iris_features` (ma trận đặc trưng của `Iris Data Set`)
1. Tính vector `iris_mean` chứa trung bình 4 đặc trưng.
2. Dùng broadcasting tạo ma trận `iris_features_centered` là ma trận `iris_features - iris_mean` (phép biến đổi dữ liệu này được gọi là phép canh giữa - centering).
3. Kiểm tra lại để thấy `iris_features_centered` có trung bình tất cả các cột là 0.

In [87]:
iris_mean = iris_features.mean(axis=0)
iris_mean

array([5.8433347, 3.054    , 3.7586665, 1.1986669], dtype=float32)

In [88]:
iris_features_centered = iris_features - iris_mean
iris_features_centered[:5]

array([[-0.7433348 ,  0.4460001 , -2.3586664 , -0.99866694],
       [-0.9433346 , -0.0539999 , -2.3586664 , -0.99866694],
       [-1.1433349 ,  0.14600015, -2.4586666 , -0.99866694],
       [-1.2433348 ,  0.046     , -2.2586665 , -0.99866694],
       [-0.8433347 ,  0.546     , -2.3586664 , -0.99866694]],
      dtype=float32)

In [89]:
np.average(iris_features_centered, axis=0) # Nếu = 0 thì sai do sai số

array([-1.4432271e-06,  1.0172526e-07, -7.6293944e-08, -3.0279159e-07],
      dtype=float32)

**Bài tập**

Từ ma trận `iris_features_centered`
1. Tính vector `iris_sd` chứa độ lệch chuẩn 4 đặc trưng.
2. Dùng broadcasting tạo ma trận `iris_features_standardized` là ma trận `iris_features_centered/iris_sd` (phép biến đổi dữ liệu này được gọi là phép tỉ lệ - scaling).
3. Kiểm tra lại để thấy `iris_features_standardized` có trung bình tất cả các cột là 0 và độ lệch chuẩn tất cả các cột là 1. 

**Bài tập**

Từ ma trận `iris_features`, tương tự Câu 1, 2
1. Biến đổi dữ liệu để tất cả các cột đều có giá trị trong khoảng [0, 1] bằng cách trừ cho min và chia cho range (range là max - min).
3. Kiểm tra lại để thấy range của các cột đều là 1.

## Các phép toán so sánh và luận lý

Các phép toán so sánh (<, ==, !=, ...) cũng được NumPy thực hiện theo từng phần tử trong mảng với ufunc.

In [90]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [91]:
x <= 5

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False])

In [92]:
np.sum(x<5)

5

In [93]:
x == 5

array([False, False, False, False, False,  True, False, False, False,
       False])

In [94]:
2*x == x**2

array([ True, False,  True, False, False, False, False, False, False,
       False])

In [95]:
x = np.arange(10).reshape(2, -1)
x

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [96]:
x % 2 == 0

array([[ True, False,  True, False,  True],
       [False,  True, False,  True, False]])

Các phép toán luận lý được nạp chồng bởi các toán tử bitwise logic: & (và), | (hoặc), ~ (phủ định), ^ (XOR).

In [97]:
x = np.arange(10).reshape(2, -1)
x

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [98]:
(x % 2 == 0) & (x < 5)

array([[ True, False,  True, False,  True],
       [False, False, False, False, False]])

In [99]:
(x % 2 == 0) | (x < 5)

array([[ True,  True,  True,  True,  True],
       [False,  True, False,  True, False]])

In [100]:
(x % 2 == 0) ^ (x < 5)

array([[False,  True, False,  True, False],
       [False,  True, False,  True, False]])

In [101]:
~(x % 2 == 0)

array([[False,  True, False,  True, False],
       [ True, False,  True, False,  True]])

Các hàm `np.count_nonzero`, `np.sum`, `np.any`, `np.all` hay được dùng để kiểm tra kết quả so sánh, luận lý.

In [102]:
x = np.arange(12).reshape(3, -1)
x

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [103]:
y = x < 5
y

array([[ True,  True,  True,  True],
       [ True, False, False, False],
       [False, False, False, False]])

In [104]:
np.count_nonzero(y)

5

In [105]:
np.sum(y)

5

In [106]:
np.sum(y, axis=0)

array([2, 1, 1, 1])

In [107]:
np.sum(y, axis=1)

array([4, 1, 0])

In [108]:
np.all(y, axis=1)

array([ True, False, False])

In [109]:
np.any(y, axis=0)

array([ True,  True,  True,  True])

Ta có thể dùng mảng bool để chọn ra các phần tử thỏa mãn điều kiện nào đó (kĩ thuật này được gọi là **masking**).

In [110]:
print(x)
print(y)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ True  True  True  True]
 [ True False False False]
 [False False False False]]


In [111]:
x[y]

array([0, 1, 2, 3, 4])

In [112]:
x[~y]

array([ 5,  6,  7,  8,  9, 10, 11])

In [113]:
x[(5 < x) & (x < 10)]

array([6, 7, 8, 9])

Ví dụ sau đếm số lượng hoa Iris thuộc giống setosa.

In [114]:
np.unique(iris_classes)

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype='<U20')

In [115]:
np.sum(iris_classes == "Iris-setosa")

50

Ví dụ sau đếm số lượng hoa Iris có sepal length lớn hơn 5.

In [116]:
np.sum(iris_features[:, 0] > 5)

118

Ví dụ sau tính trung bình sepal length của giống setosa.

In [117]:
iris_features[iris_classes == "Iris-setosa", 0].mean()

5.006

**Bài tập**

Đếm số lượng hoa Iris giống virginica có sepal width lớn hơn 3.

**Bài tập**

Cho biết min, max, mean của sepal width của các hoa Iris giống virginica.

## Truy cập các phần tử của mảng bằng danh sách các chỉ số

Ta đã thấy cách dùng chỉ số (index), trượt (slice) và mảng luận lý (mask) để truy cập các phần tử của mảng `ndarray`.

In [118]:
x = 10*np.arange(10)

print(x)
print(x[5])
print(x[:5])
print(x[x > 50])

[ 0 10 20 30 40 50 60 70 80 90]
50
[ 0 10 20 30 40]
[60 70 80 90]


Ta cũng có thể dùng mảng (hay danh sách) các chỉ số để lấy về tập các phần tử (kỹ thuật này được gọi là **fancy indexing**).

In [119]:
print(x)
print([x[3], x[5], x[7], x[9]])
print(x[[3, 5, 7, 9]])
print(x[[i for i in range(3, 10, 2)]])

[ 0 10 20 30 40 50 60 70 80 90]
[30, 50, 70, 90]
[30 50 70 90]
[30 50 70 90]


Mảng kết quả trả về có dạng như mảng index.

In [120]:
ind = np.array([
    [0, 3],
    [2, 4]
])

print(ind)
print(x[ind])

[[0 3]
 [2 4]]
[[ 0 30]
 [20 40]]


Fancy index cũng có thể dùng trên mảng nhiều chiều. Khi đó ta cần danh sách chỉ số trên từng trục.

In [121]:
x = np.arange(12).reshape(3, -1)
x

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [122]:
row_ind = [0, 0, -1]
col_ind = [1, 0, -1]
x[row_ind, col_ind]

array([ 1,  0, 11])

Ta cũng có thể dùng kết hợp tất cả các kĩ thuật trên (index, slice, mask, fancy index).

In [123]:
x = np.arange(20).reshape(4, -1)
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [124]:
x[:, [0, -1, 2]]

array([[ 0,  4,  2],
       [ 5,  9,  7],
       [10, 14, 12],
       [15, 19, 17]])

In [125]:
x[[0, -1],:] = 0
x

array([[ 0,  0,  0,  0,  0],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [ 0,  0,  0,  0,  0]])

**Bài tập**

Cho mảng `a` vuông kích thước 5x5 như bên dưới
1. Dùng fancy index đặt tất cả các phần tử trên đường chéo chính thành 0.
2. Nghĩ các cách Pythonic khác nhau để thực hiện công việc 1.

In [126]:
a = np.arange(25).reshape(5, -1)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

## Một vài thao tác đặc biệt trên mảng

### Sắp xếp mảng bằng hàm `np.sort` và `np.argsort`

In [127]:
x = np.random.randint(0, 100, 10)
x

array([50, 90, 68, 37, 18, 82, 98, 46, 89, 11])

In [128]:
print(np.sort(x))
print(np.argsort(x))
print(x)
print(x[np.argsort(x)])

[11 18 37 46 50 68 82 89 90 98]
[9 4 3 7 0 2 5 8 1 6]
[50 90 68 37 18 82 98 46 89 11]
[11 18 37 46 50 68 82 89 90 98]


In [129]:
x.sort()
print(x)

[11 18 37 46 50 68 82 89 90 98]


In [130]:
x = np.random.randint(0, 100, (4, 5))
x

array([[75, 75, 34,  2, 62],
       [92, 21, 94, 75, 24],
       [81, 48, 85, 24, 98],
       [32, 74, 83, 34, 76]])

In [131]:
np.sort(x)

array([[ 2, 34, 62, 75, 75],
       [21, 24, 75, 92, 94],
       [24, 48, 81, 85, 98],
       [32, 34, 74, 76, 83]])

In [132]:
np.sort(x, axis=0)

array([[32, 21, 34,  2, 24],
       [75, 48, 83, 24, 62],
       [81, 74, 85, 34, 76],
       [92, 75, 94, 75, 98]])

In [133]:
np.sort(x, axis=1)

array([[ 2, 34, 62, 75, 75],
       [21, 24, 75, 92, 94],
       [24, 48, 81, 85, 98],
       [32, 34, 74, 76, 83]])

### Kiểm tra các giá trị duy nhất trong mảng với `np.unique`

In [134]:
x = np.random.randint(0, 3, 10)
x

array([0, 2, 0, 0, 1, 1, 0, 2, 2, 2])

In [135]:
np.unique(x)

array([0, 1, 2])

In [136]:
_, ind = np.unique(x, return_index=True)
ind

array([0, 4, 1])

In [137]:
_, counts = np.unique(x, return_counts=True)
counts

array([4, 2, 4])

### Chuyển vị (transpose) ma trận (biến dòng thành cột, cột thành dòng) với thuộc tính `.T`

In [138]:
x = np.random.randint(0, 10, (3, 4))
x

array([[1, 5, 4, 4],
       [0, 4, 4, 3],
       [4, 8, 4, 5]])

In [139]:
x.T

array([[1, 0, 4],
       [5, 4, 8],
       [4, 4, 4],
       [4, 3, 5]])

In [140]:
x.reshape(4, 3)

array([[1, 5, 4],
       [4, 0, 4],
       [4, 3, 4],
       [8, 4, 5]])

### Nhân ma trận (matrix multiplication) với toán tử `@` hay hàm `np.dot`

In [141]:
a = np.arange(6).reshape(2, -1)
print(a)
print(a.T)

[[0 1 2]
 [3 4 5]]
[[0 3]
 [1 4]
 [2 5]]


In [142]:
a * a

array([[ 0,  1,  4],
       [ 9, 16, 25]])

In [143]:
a @ a.T

array([[ 5, 14],
       [14, 50]])

In [144]:
np.dot(a, a.T)

array([[ 5, 14],
       [14, 50]])

In [145]:
a.dot(a.T)

array([[ 5, 14],
       [14, 50]])

### Đảo ngược mảng với `np.flip`

In [146]:
x = np.random.randint(0, 10, (3, 4))
x

array([[7, 6, 0, 4],
       [1, 2, 4, 2],
       [6, 7, 9, 0]])

In [147]:
np.flip(x)

array([[0, 9, 7, 6],
       [2, 4, 2, 1],
       [4, 0, 6, 7]])

In [148]:
np.flip(x, axis=0)

array([[6, 7, 9, 0],
       [1, 2, 4, 2],
       [7, 6, 0, 4]])

In [149]:
np.flip(x, axis=1)

array([[4, 0, 6, 7],
       [2, 4, 2, 1],
       [0, 9, 7, 6]])

In [150]:
print(x)
x[:, -1] = np.flip(x[:, -1])
print(x)

[[7 6 0 4]
 [1 2 4 2]
 [6 7 9 0]]
[[7 6 0 0]
 [1 2 4 2]
 [6 7 9 4]]


## Đọc thêm tài liệu NumPy

1. [NumPy quickstart](https://numpy.org/doc/stable/user/quickstart.html)


1. [NumPy: the absolute basics for beginners](https://numpy.org/doc/stable/user/absolute_beginners.html)

1. [NumPy fundamentals](https://numpy.org/doc/stable/user/basics.html)

1. [NumPy Tutorials](https://numpy.org/numpy-tutorials/features.html)